In [ ]:
# Name: Cheryl Cook | Student ID: 300587497 | ECS Username: cookcher

# Part 2: Dense Tracking Using Optical Flow

import zipfile # For extracting 'data.zip'
import os # For image file handling
import cv2 # for Optical Flow
import numpy as np # For calculations
import matplotlib.pyplot as plt  # For visualisations

# In order for extraction below to work, need to first upload 'data.zip' to Files
# Extracting 'data.zip'
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Both tasks (optical flow calculation and visualisation) completed in this cell

# Output folder to store every 5th frame
opticalflow_results = 'OpticalFlow_Results/'
os.makedirs(opticalflow_results, exist_ok=True) # creating folder if it does not yet exist

for i in range(0,69): # 0 to 68
  f1_path = os.path.join(f'data/frame_{i}.jpg')
  f2_path = os.path.join(f'data/frame_{i+1}.jpg')

  # Loading the frames
  frame1_BGR = cv2.imread(f1_path)
  frame2_BGR = cv2.imread(f2_path)

  # Converting the frames to grayscale for Optical Flow
  frame1_gray = cv2.cvtColor(frame1_BGR, cv2.COLOR_BGR2GRAY)
  frame2_gray = cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2GRAY)

  # If either frame is not loaded, raise an error
  if frame1_gray is None or frame2_gray is None:
    raise ValueError("Error loading frame. Check that the file path exists.")

  # Task 1: Optical Flow Calculation (using a dense grid of points and Lucas-Kanade method)

  # This section of code is mostly from the Week 6 Tutorial
  # Step 1: Creating a grid of points (dense sampling)
  step_size = 20 # increased step size from tutorial's 10 to 20
  f1_h, f1_w = frame1_gray.shape
  grid_x, grid_y = np.meshgrid(np.arange(0, f1_w, step_size), np.arange(0, f1_h, step_size))
  points = np.vstack((grid_x.ravel(), grid_y.ravel())).T
  # This step was suggested by Jennifer to enable all points to be passed to cv2.calcOpticalFlowPyrLK at once
  points_reshaped = np.array(points, dtype=np.float32).reshape(-1,1,2)

  # Step 2: Calculating optical flow for each point using Lucas-Kanade method
  lk_params = dict(winSize=(15, 15), maxLevel=4)
  # Calculating optical flow for each point
  # Passing the whole points array at once was suggested by Jennifer to make the code run faster
  flow_points, status, err = cv2.calcOpticalFlowPyrLK(frame1_gray, frame2_gray, points_reshaped, None, **lk_params)
  # Want to retain only the points that were successfully tracked
  # For these next two lines, referred to https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
  old_points = points_reshaped[status==1]
  new_points = flow_points[status==1]

  # Task 2: Visualisation

  # Drawing flow arrows
  for (x1, y1), (x2, y2) in zip(old_points, new_points):
    cv2.arrowedLine(frame2_BGR, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1, tipLength=0.4)

  # Visualizing the current frame with the flow arrows
  plt.figure(figsize=(12, 6))
  plt.imshow(cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title(f"Dense Optical Flow (Lucas-Kanade): frame_{i} -> frame_{i+1}")
  plt.show()

  # Saving every fifth frame to 'OpticalFlow_Results.zip'
  if (i+1) % 5 == 0:
      filename = os.path.join(opticalflow_results, f'frame_{i+1}.jpg')
      cv2.imwrite(filename, frame2_BGR)

## The cell below is only needed if running the code in Google Colab rather than locally on Jupyter Notebook.

In [ ]:
# Downloading the saved 'OpticalFlow_Results' from Google Colab to my local machine

from google.colab import files
import shutil

# Converting the file into a zip file
shutil.make_archive('OpticalFlow_Results', 'zip', 'OpticalFlow_Results')

# Downloading the zip file to my local machine
files.download('OpticalFlow_Results.zip')